In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

from Constants.Paths import *
from Constants.Labels import *
from Unpacking.PrepareAudioFiles import prepare_audio_files
from Preprocessing.GenerateSpectrograms import generate_all_spectrograms
from SpectrogramLoading import *
from Models.TrainingHistory import TrainingHistory
from Models.CnnModel import CnnModel
from Models.WandbDetails import WandbDetails
from Models.InputPadding import pad_to_length
from Models.HistoryPlots import plot_loss_history, plot_accuracy_history

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
prepare_audio_files()
generate_all_spectrograms(backend="soundfile")

In [ ]:
train_paths, val_paths = get_divided_paths_with_labels()

random.seed(42)
random.shuffle(train_paths)
random.shuffle(val_paths)

train = [load_spectrogram_for_path(path_with_label) for path_with_label in train_paths]
validation = [load_spectrogram_for_path(path_with_label) for path_with_label in val_paths]

In [ ]:
X_train, y_train, label_indexes = spectrograms_to_x_y(train)
X_validation, y_validation, _ = spectrograms_to_x_y(validation, label_indexes)

max_length = max(x.shape[1] for x in [*X_validation, *X_train])
X_validation = pad_to_length(X_validation, max_length)
X_train = pad_to_length(X_train, max_length)

In [ ]:
from Models.TransformerModel import TransformerModel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix
)

In [ ]:
# Testing Transformer

def train_and_evaluate_transformer(params):
    model = TransformerModel(
        classes=labels,
        embedding_dimension=params.get("embedding_dimension", 512),
        num_attention_heads=params.get("num_attention_heads", 8),
        num_encoder_layers=params.get("num_encoder_layers", 6),
        num_decoder_layers=params.get("num_decoder_layers", 6),
        dim_feedforward=params.get("dim_feedforward", 2048),
        dropout=params.get("dropout", 0.1),
        learning_rate=params.get("learning_rate", 1e-4),
        lr_decay=params.get("lr_decay", 0.0),
        beta_1=params.get("beta_1", 0.9),
        beta_2=params.get("beta_2", 0.98),
        eps=params.get("eps", 1e-9),
        print_every=None,
        validate_every=1,
        seed=42
    )
    
    model.train((X_train, y_train), (X_validation, y_validation), epochs=5, batch_size=32)
    
    y_pred_classes = model.predict(X_validation)
    
    acc = accuracy_score(y_validation, y_pred_classes)
    precision, recall, f1_score, _ = precision_recall_fscore_support(
        y_validation, y_pred_classes, average=None, labels=list(range(len(labels)))
    )
    cm = confusion_matrix(y_validation, y_pred_classes)
    history = model.get_history()
    
    return {
        "params": params,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "confusion_matrix": cm,
        "history": history
    }

def save_results_to_json(results, filename):
    serializable_results = []
    
    for result in results:
        serializable_result = {
            "params": result["params"],
            "accuracy": result["accuracy"],
            "precision": result["precision"].tolist(),
            "recall": result["recall"].tolist(),
            "f1_score": result["f1_score"].tolist(),
            "confusion_matrix": result["confusion_matrix"].tolist(),
            "history": {
                "loss": result["history"].loss,
                "val_loss": result["history"].val_loss,
                "accuracy": result["history"].accuracy,
                "val_accuracy": result["history"].val_accuracy,
            }
        }
        serializable_results.append(serializable_result)
    
    with open(filename, "w") as f:
        json.dump(serializable_results, f, indent=4)

param_grid = [
    {**base_params, "embedding_dimension": 256},
    {**base_params, "embedding_dimension": 512},
    {**base_params, "embedding_dimension": 1024},
    
    {**base_params, "num_attention_heads": 4},
    {**base_params, "num_attention_heads": 8},
    {**base_params, "num_attention_heads": 16},
    
    {**base_params, "dropout": 0.1},
    {**base_params, "dropout": 0.2},
    {**base_params, "dropout": 0.3},
    
    {**base_params, "learning_rate": 1e-3},
    {**base_params, "learning_rate": 1e-4},
    {**base_params, "learning_rate": 1e-5},
]

results = []

for params in param_grid:
    result = train_and_evaluate_transformer(params)
    results.append(result)

save_results_to_json(results, "transformer_experiments_results.json")

for i, result in enumerate(results):
    print(f"\nModel {i+1}")
    print(f"Params: {result['params']}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Precision: {result['precision']}")
    print(f"Recall: {result['recall']}")
    print(f"F1-Score: {result['f1_score']}")
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(result["confusion_matrix"], annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix for Model {i+1}')
    plt.show()

In [ ]:
# Testing CNN

def train_and_evaluate_cnn(params):
    model = CnnModel(
        classes=labels,
        learning_rate=params.get("learning_rate", 1e-3),
        lr_decay=params.get("lr_decay", 1e-4),
        beta_1=params.get("beta_1", 0.9),
        beta_2=params.get("beta_2", 0.999),
        eps=params.get("eps", 1e-8),
        classifier_dropout_1=params.get("classifier_dropout_1", 0.3),
        classifier_dropout_2=params.get("classifier_dropout_2", 0.3),
        classifier_dropout_3=params.get("classifier_dropout_3", 0.1),
        print_every=None,
        validate_every=1,
        seed=42
    )
    
    model.train((X_train, y_train), (X_validation, y_validation), epochs=5, batch_size=32)
    
    y_pred_classes = model.predict(X_validation)
    
    acc = accuracy_score(y_validation, y_pred_classes)
    precision, recall, f1_score, _ = precision_recall_fscore_support(
        y_validation, y_pred_classes, average=None, labels=list(range(len(labels)))
    )
    cm = confusion_matrix(y_validation, y_pred_classes)
    history = model.get_history()
    
    return {
        "params": params,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "confusion_matrix": cm,
        "history": history
    }

def save_results_to_json(results, filename):
    serializable_results = []
    
    for result in results:
        serializable_result = {
            "params": result["params"],
            "accuracy": result["accuracy"],
            "precision": result["precision"].tolist(),
            "recall": result["recall"].tolist(),
            "f1_score": result["f1_score"].tolist(),
            "confusion_matrix": result["confusion_matrix"].tolist(),
            "history": {
                "loss": result["history"].loss,
                "val_loss": result["history"].val_loss,
                "accuracy": result["history"].accuracy,
                "val_accuracy": result["history"].val_accuracy,
            }
        }
        serializable_results.append(serializable_result)
    
    with open(filename, "w") as f:
        json.dump(serializable_results, f, indent=4)

# Definicja siatki parametrów
param_grid = [
    {**base_params, "classifier_dropout_1": 0.2, "classifier_dropout_2": 0.2, "classifier_dropout_3": 0.1},
    {**base_params, "classifier_dropout_1": 0.3, "classifier_dropout_2": 0.3, "classifier_dropout_3": 0.1},
    {**base_params, "classifier_dropout_1": 0.4, "classifier_dropout_2": 0.4, "classifier_dropout_3": 0.2},
    
    {**base_params, "learning_rate": 1e-3},
    {**base_params, "learning_rate": 1e-4},
    {**base_params, "learning_rate": 5e-5},
]

results = []

for params in param_grid:
    result = train_and_evaluate_cnn(params)
    results.append(result)

save_results_to_json(results, "cnn_experiments_results.json")

for i, result in enumerate(results):
    print(f"\nModel {i+1}")
    print(f"Params: {result['params']}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Precision: {result['precision']}")
    print(f"Recall: {result['recall']}")
    print(f"F1-Score: {result['f1_score']}")
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(result["confusion_matrix"], annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix for Model {i+1}')
    plt.show()
